In [16]:
import pandas as pd
import numpy as np
from tf_idf_functions import calculate_tfidf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
tqdm.pandas()

In [17]:
def split_test_sets(df, test_size=0.2, temporal_test_size=75, random_state=42):
    """
    Splits the dataset into training, validation, temporal test, and thrower test sets.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing throw data.
    - test_size (float, optional): The proportion of the dataset to include in the validation set. Default is 0.2.
    - temporal_test_size (int, optional): The number of games to include in the temporal test set. Default is 75.
    - random_state (int, optional): Random seed for reproducibility. Default is 42.
    
    Returns:
    - train_df (pd.DataFrame): Training set.
    - val_df (pd.DataFrame): Validation set.
    - temporal_test_df (pd.DataFrame): Temporal test set.
    - thrower_test_df (pd.DataFrame): Hold-out test set for specific throwers.
    """
    # Select 50 throwers with > 200 throws for hold-out-testing
    throwers = df.groupby('thrower').thrower.count()
    hold_out_throwers = np.array(throwers[throwers > 200].index)
    np.random.seed(random_state)
    hold_out_throwers = np.random.choice(hold_out_throwers, size=50, replace=False)
    thrower_test_df = df[df.thrower.isin(hold_out_throwers)].copy()
    df_filtered = df[~df.thrower.isin(hold_out_throwers)].copy()
    
    # Extract date and year
    df_filtered.loc[:,'gameDate'] = pd.to_datetime(df_filtered.loc[:,'gameID'].str[:10], format='%Y-%m-%d')
    df_filtered.loc[:,'year'] = df_filtered.loc[:,'gameDate'].dt.year
    df_sorted = df_filtered.sort_values(by='gameDate')
    
    # Select 75 most recent games for hold-out-testing 
    unique_game_ids = df_sorted.loc[:,'gameID'].unique()
    temporal_test_ids = unique_game_ids[-temporal_test_size:]
    
    # In remeaining data, split into test and train. Make sure the hold out testing set is balanced between years
    remaining_ids = unique_game_ids[:-temporal_test_size]
    total_remaining = len(remaining_ids)
    num_games_per_year = int(total_remaining * test_size / 4)
    remaining_df = df_sorted[df_sorted.loc[:,'gameID'].isin(remaining_ids)]
    val_games = []
    for year, group in remaining_df.groupby('year'):
        sampled_games = group.sample(n=num_games_per_year, random_state=random_state)
        val_games.append(sampled_games.loc[:,'gameID'].unique())
    
    val_ids = np.concatenate(val_games)
    

    train_ids = [game_id for game_id in remaining_ids if game_id not in val_ids]
    train_df = df_filtered[df_filtered.loc[:,'gameID'].isin(train_ids)]
    val_df = df_filtered[df_filtered.loc[:,'gameID'].isin(val_ids)]
    temporal_test_df = df_filtered[df_filtered.loc[:,'gameID'].isin(temporal_test_ids)]
    
    return train_df, val_df, temporal_test_df, thrower_test_df


def calculate_throw_direction_percentages(df):
    """
    Calculates the percentage of throws categorized as forward, sideways, or backward for each thrower.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing throw data, with 'thrower' and 'throw_distance' columns.
    
    Returns:
    - pd.DataFrame: Original DataFrame with new columns for throw direction percentages 
                    ('backward_percentage', 'sideways_percentage', 'forward_percentage').
    """

    direction_counts = df.groupby(['thrower', 'direction_category']).size().unstack(fill_value=0)
    direction_counts['total_throws'] = direction_counts.sum(axis=1)
    direction_percentages = direction_counts.div(direction_counts['total_throws'], axis=0) * 100


    direction_percentages = direction_percentages.rename(columns={
        'Backwards': 'backward_percentage',
        'Sideways': 'sideways_percentage',
        'Forward': 'forward_percentage'
    })
    direction_percentages = direction_percentages.reset_index()
    df = df.merge(direction_percentages[['thrower', 'backward_percentage', 'sideways_percentage', 'forward_percentage']],
                   on='thrower', how='left')

    return df


In [18]:
def initialize_data(filepath='./data/processed/all_games_0926.csv'):
    """
    Initializes the dataset by loading a CSV file, processing the data to calculate various columns
    like goal, outcome, point_outcome, throw_distance, and completion. The function handles missing values
    and adjusts the 'times' column based on certain time limits.
    
    Parameters:
    - filepath (str): The path to the CSV file containing the game data.
    
    Returns:
    - pd.DataFrame: The processed DataFrame with calculated columns.
    """
    df = pd.read_csv(filepath)
    df = df.dropna(subset=['thrower_x', 'thrower_y'])
    df.loc[:,'goal'] = (df.loc[:,'receiver_y'] > 100) & (df.loc[:,'turnover'] == 0)
    df.loc[:,'outcome'] = 0  # 0 if possession ended without a goal otherwise 1
    df.loc[:,'point_outcome'] = 0 # 0 if point ended without a goal otherwise 1
    df.loc[:,'throw_distance'] = np.sqrt((df.loc[:,'receiver_x'] - df.loc[:,'thrower_x'])**2 + (df.loc[:,'receiver_y'] - df.loc[:,'thrower_y'])**2)
    df.loc[:,'completion'] = 1 - df.loc[:,'turnover']
    df.loc[:,'gameDate'] = pd.to_datetime(df.loc[:,'gameID'].str[:10], format='%Y-%m-%d')
    df.loc[:,'year'] = df.loc[:,'gameDate'].dt.year


    df['prev_receiver'] = df['receiver'].shift(1)
    df['prev_thrower'] = df['thrower'].shift(1)
    hockey_assists_df = df[
        (df['receiver_y'] > 100) & (df['turnover'] == 0) & 
        (df['thrower'] == df['prev_receiver'])
    ]

    df['hockey_assist'] = 0  
    df.loc[hockey_assists_df.index - 1, 'hockey_assist'] = 1  
    df.drop(['prev_receiver', 'prev_thrower'], axis=1, inplace=True)

    df.times = (df.times / 60)
    df.loc[df['times'] < 0, 'times'] = df.loc[df['times'] < 0, 'times'] + 5
    while len(df.times[df.times > 12]) > 0:
        df.loc[df['times'] > 12, 'times'] = df.loc[df['times'] > 12, 'times'] - 12

    for _, group in df.groupby(['gameID', 'home_team_score', 'away_team_score', 'possession_num', 'game_quarter']):
        last_throw = group.iloc[-1]  
        if last_throw['receiver_y'] > 100 and last_throw['turnover'] == 0:
            df.loc[group.index, 'outcome'] = 1  
    for _, group in df.groupby(['gameID', 'home_team_score', 'away_team_score', 'game_quarter']):
        last_throw = group.iloc[-1]  # Get the last row in the group
        if last_throw['receiver_y'] > 100 and last_throw['turnover'] == 0:
            mask = group['is_home_team'] == last_throw['is_home_team']
            df.loc[group[mask].index, 'point_outcome'] = 1
    return df

In [19]:
def get_completion_percentage(df, test_dfs=None):
    """
    Calculate the completion percentage for each thrower and add it to the DataFrame.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing throw data with a 'thrower' and 'completion' column.
    - test_dfs (list of pd.DataFrame, optional): List of test DataFrames to which the completion percentage will also be added.
    
    Returns:
    - train_df (pd.DataFrame): DataFrame with completion percentage added.
    - test_dfs_final (list of pd.DataFrame): List of test DataFrames with completion percentage added (if provided).
    """
    
    # Group by 'thrower' and calculate successful completions and total throws
    completion_stats = df.groupby('thrower').agg(
        successful_completions=('completion', 'sum'),  # Total successful completions per thrower
        total_throws=('completion', 'count')  # Total number of throws per thrower
    ).reset_index()

    #  Calculate completion percentage for each thrower
    completion_stats['completion_percentage'] = (
        completion_stats['successful_completions'] / completion_stats['total_throws']
    ) * 100
    new_rows = completion_stats[['thrower', 'completion_percentage']].rename(
        columns={'completion_percentage': 'thrower_completion_percentage'}
    )
    train_df = df.merge(new_rows, on='thrower', how='left').sort_values('thrower_completion_percentage')

    test_dfs_final = []
    if test_dfs is not None:
        # For each test DataFrame, merge completion percentage and fill missing values with the median
        for test_df in test_dfs:
            test_df = test_df.merge(new_rows, on='thrower', how='left').sort_values('thrower_completion_percentage')
            test_df.thrower_completion_percentage = test_df.thrower_completion_percentage.fillna(
                test_df.thrower_completion_percentage.median()
            )
            test_dfs_final.append(test_df)

    return train_df, test_dfs_final


In [20]:
def one_hot_encode_categories(dfs):
    """
    One-hot encode the 'distance_category' and 'direction_category' columns for each DataFrame in a list.
    
    Parameters:
    - dfs (list of pd.DataFrame): List of DataFrames containing 'distance_category' and 'direction_category' columns.
    
    Returns:
    - dfs_final (list of pd.DataFrame): List of DataFrames with new one-hot encoded columns for each unique category in 'distance_category' and 'direction_category'.
    """
    
    dfs_final = [] 
    
    for df in dfs: 
        for category in df['distance_category'].unique():
            df[f'distance_{category}'] = df['distance_category'] == category
        for category in df['direction_category'].unique():
            df[f'direction_{category}'] = df['direction_category'] == category
        
        dfs_final.append(df)
    
    return dfs_final


In [21]:
def calculate_throw_direction_percentages(df, test_dfs=None):
    """
    Calculate the percentage of throws in each direction (backward, sideways, forward) for each thrower.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing throw data with 'thrower' and 'direction_category' columns.
    - test_dfs (list of pd.DataFrames, optional): A list of DataFrames to apply the same percentage calculations to.

    Returns:
    - pd.DataFrame: Original DataFrame with added columns for backward, sideways, and forward throw percentages.
    - list of pd.DataFrames: (if test_dfs provided) List of test DataFrames with added percentage columns.
    """

    direction_counts = df.groupby(['thrower', 'direction_category']).size().unstack(fill_value=0)
    direction_counts['total_throws'] = direction_counts.sum(axis=1)

    # Convert the counts to percentages by dividing by the total throws and multiply by 100
    direction_counts['backward_percentage'] = (direction_counts['backward'] / direction_counts['total_throws']) * 100
    direction_counts['sideways_percentage'] = (direction_counts['sideways'] / direction_counts['total_throws']) * 100
    direction_counts['forward_percentage'] = (direction_counts['forward'] / direction_counts['total_throws']) * 100

    direction_counts = direction_counts.reset_index()
    df = df.merge(direction_counts[['thrower', 'backward_percentage', 'sideways_percentage', 'forward_percentage']],
                  on='thrower', how='left')

    # If test_dfs are provided, apply the same process to each test DataFrame
    test_dfs_final = []
    if test_dfs is not None:
        for test_df in test_dfs:
            test_df = test_df.merge(direction_counts[['thrower', 'backward_percentage', 'sideways_percentage', 'forward_percentage']],
                                    on='thrower', how='left')
            test_dfs_final.append(test_df)
    
    return df, test_dfs_final

In [22]:
filepath='../data/all_games_0926.csv'
df = initialize_data(filepath)

train_df, test_df_random, test_df_time, test_df_thrower = split_test_sets(df, random_state=0, test_size=0.4)
train_df, test_dfs = get_completion_percentage(df, test_dfs=[test_df_random, test_df_time, test_df_thrower])
train_df, test_dfs = calculate_tfidf(train_df, test_dfs)
train_df, test_dfs = calculate_throw_direction_percentages(train_df, test_dfs)
train_df, test_df_random, test_df_time, test_df_thrower = one_hot_encode_categories([train_df] + test_dfs)
joblib.dump({'train_df':train_df, 'test_df_random':test_df_random, 'test_df_time':test_df_time, 'test_df_thrower':test_df_thrower}, '../data/processed/data_splits_1003.jblb')

['../data/processed/data_splits_1003.jblb']

In [23]:
filepath='../data/all_games_0926.csv'
df = initialize_data(filepath)
df, _ = get_completion_percentage(df)
df, _ = calculate_tfidf(df)
df, _ = calculate_throw_direction_percentages(df)
df = one_hot_encode_categories([df])[0]

joblib.dump({'df':df}, '../data/processed/data_1003.jblb')

['../data/processed/data_1003.jblb']